In [1]:
import tensorflow as tf
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import numpy as np
from tensorflow.python.client import device_lib
from keras.models import load_model
import gc
import onnx
import onnxruntime as rt

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.scaler import *
from mclLib.ftploader import *
from mclLib.modelpostfix import *
from mclLib.tester import *
from mclLib.server_info import *

In [2]:
print(device_lib.list_local_devices())

engine = create_engine(mysql_server_uri)
conn = engine.connect()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5442515185266207108
xla_global_id: -1
]


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-03-01')
br = br_full_data[get_filter]

In [5]:
''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y = br['fMaxPowerAfterBuyWhile30'].to_numpy()

y_test = y
y_predict = []

modelTester = ModelTester(engine, conn)

In [6]:
# 테스트할 모델

scale_yes_model_names = [] 

scale_no_model_names = [ 
    'REG_RF_pow30_n20_md10'
]


In [7]:
ftp= FtpLoader(ftp_ip, ftp_port, ftp_id, ftp_pw)

In [8]:
for idx, mn in enumerate(scale_no_model_names):
    #ftp.download(mn+onnx_, onnx_path, just_onnx)
    
    print(onnx_path + mn + onnx_)
    x_datas = X

    # onnx_model = onnx.load(onnx_path + mn + onnx_)
    sess = rt.InferenceSession(onnx_path + mn + onnx_)
    res = sess.run(['output'],{'input': x_datas.astype(dtype=np.float32)})
    y_predict.append(res[0])
    
    del sess
    gc.collect()

C:/Users/MJ/source/repos/AtoGenerator/임시저장소/onnx/REG_RF_pow30_n20_md10.onnx


In [12]:
#################################
############## test #############

# suc_crit = 0.8 # 1이라 판정할 값의 기준
# fl_crit = 0.5  # 0이라 판정할 값의 기준

suc_ratio = 0.65 # 모델들에서 1이라 종합할 비율
fail_ratio = .9 # 모델들에서 0이라 종합할 비율

_, fal_li = testRegression(y_test, y_predict, crit_val=0.05, suc_pass_ratio=suc_ratio, fail_pass_ratio=fail_ratio)


회귀평가를 시작합니다...
len of models :  1
suc_line : 1 (0.65)
fail_line : 1 (0.9)
crit value :  0.05

총량 :  153903
0 :  21683 , 비율 :  14.08874420901477 (%)
1 :  132220 , 비율 :  85.91125579098524 (%)

============ predict 0 =============
총 횟수 :  3809
실제 0 :  1198
실제 1 :  2611
정답비율 :  31.45182462588606 (%)

============ predict 1 =============
총 횟수 :  150094
실제 1 :  129609
실제 0 :  20485
정답비율 :  86.35188615134516 (%)

